**Import Necessary Libraries**

In [ ]:
!pip install flask --quiet
!pip install flask-ngrok --quiet
print("Completed!")

Completed!


In [ ]:
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py


In [ ]:
%pip install -qq git+https://github.com/ShivamShrirao/diffusers
%pip install -q -U --pre triton
%pip install -q accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.4

**Setup and Installation of Ngrok**

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz

--2023-04-19 06:45:39--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 52.202.168.65, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13856790 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.21M  40.9MB/s    in 0.3s    

2023-04-19 06:45:40 (40.9 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13856790/13856790]



In [ ]:
!tar -xvf /content/ngrok-stable-linux-amd64.tgz

ngrok


In [ ]:
!./ngrok authtoken 2OMDhKsGjanAyaQOII5qc4VLT8o_ia4KArPJP34ixF6DA6zD

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!pip install diffusers transformers --quiet

In [ ]:
!pip install accelerate --quiet

**Creating Stable diffusion model Using Python Flask Module**

In [ ]:
from flask import send_file
from flask import Flask
from flask import request
from flask_ngrok import run_with_ngrok
from PIL import Image
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, StableDiffusionInpaintPipeline, \
                      StableDiffusionImg2ImgPipeline, UniPCMultistepScheduler
from diffusers import DDIMScheduler, StableDiffusionControlNetPipeline, ControlNetModel

from matplotlib import pyplot as plt
import json
from natsort import natsorted
from glob import glob
import os
import numpy as np
import cv2

In [ ]:
from base64 import b64encode

import numpy
import torch
from diffusers import AutoencoderKL, LMSDiscreteScheduler, UNet2DConditionModel
from huggingface_hub import notebook_login

# For video display:
from IPython.display import HTML
from matplotlib import pyplot as plt
from pathlib import Path
from PIL import Image
from torch import autocast
from torchvision import transforms as tfms
from tqdm.auto import tqdm
from transformers import CLIPTextModel, CLIPTokenizer, logging

torch.manual_seed(1)
if not (Path.home()/'.huggingface'/'token').exists(): notebook_login()

# Supress some unnecessary warnings when loading the CLIPTextModel
logging.set_verbosity_error()

# Set device
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:


# Load the autoencoder model which will be used to decode the latents into image space. 
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")

# Load the tokenizer and text encoder to tokenize and encode the text. 
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

# The UNet model for generating the latents.
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet")

# The noise scheduler
scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)


def pil_to_latent(input_im):
    # Single image -> single latent in a batch (so size 1, 4, 64, 64)
    with torch.no_grad():
        latent = vae.encode(tfms.ToTensor()(input_im).unsqueeze(0).to(torch_device)*2-1) # Note scaling
    return 0.18215 * latent.latent_dist.sample()

def latents_to_pil(latents):
    # bath of latents -> list of images
    latents = (1 / 0.18215) * latents
    with torch.no_grad():
        image = vae.decode(latents).sample
    image = (image / 2 + 0.5).clamp(0, 1)
    image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
    images = (image * 255).round().astype("uint8")
    pil_images = [Image.fromarray(image) for image in images]
    return pil_images

# To the GPU we go!
vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device);
height = 512                        # default height of Stable Diffusion
width = 512                         # default width of Stable Diffusion
num_inference_steps = 50            # Number of denoising steps
guidance_scale = 8               # Scale for classifier-free guidance
generator = torch.manual_seed(32)   # Seed generator to create the inital latent noise
batch_size = 1
blue_loss_scale = 200 


/usr/local/lib/python3.9/dist-packages/safetensors/torch.py:99: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:
/usr/local/lib/python3.9/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/usr/local/lib/python3.9/dist-packages/torch/storage.py:899: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This shoul

In [ ]:
# save_to_gdrive = True
# from google.colab import drive
# if save_to_gdrive:
#   drive.mount('/content/drive')

OUTPUT_DIR = "stable_diffusion_weights/output" 
OUTPUT_DIR = "/content/" + OUTPUT_DIR

!mkdir -p $OUTPUT_DIR

instance_prompt = "photo of zwx toy" #@param {type:"string"}
class_prompt =  "photo of a toy" #@param {type:"string"}
training_steps = 800 #@param {type:"integer"}
learning_rate = 1e-6 #@param {type:"number"}

concepts_list = [
    {
        "instance_prompt":      instance_prompt,
        "class_prompt":         class_prompt,
        "instance_data_dir":    "/content/data/instance",
        "class_data_dir":       "/content/data/class"
    }
]
for c in concepts_list:
    os.makedirs(c["instance_data_dir"], exist_ok=True)

with open("concepts_list.json", "w") as f:
    json.dump(concepts_list, f, indent=4)


prompt = "oil painting of zwx toy in style of van gogh"
negative_prompt = ""
num_samples = 2
guidance_scale = 7.5
num_inference_steps = 30 
height = 512 
width = 512 
seed = 100 


In [ ]:
app = Flask(__name__) #app name
run_with_ngrok(app)

@app.route('/dreambooth', methods = ['POST'])
def dream_booth():
  ############## Edit this section to customize parameters
  for image in request.files.values():
    image.save('./data/instance/'+image.filename)
    print(image)

  !python3 train_dreambooth.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --output_dir=$OUTPUT_DIR \
  --revision="fp16" \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=1337 \
  --resolution=512 \
  --train_batch_size=1 \
  --train_text_encoder \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=$learning_rate \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=50 \
  --sample_batch_size=4 \
  --max_train_steps=$training_steps \
  --save_interval=10000 \
  --save_sample_prompt="$instance_prompt" \
  --concepts_list="concepts_list.json"

  return "Initiated Training"
 
@app.route('/dreambooth_inference', methods = ['POST'])
def dream_booth_inference():
  weightdirs = natsorted(glob(OUTPUT_DIR + os.sep + "*"))
  if len(weightdirs) == 0:
    raise KeyboardInterrupt("No training weights directory found")
  WEIGHTS_DIR = weightdirs[-1]
  model_path = WEIGHTS_DIR             # If you want to use previously trained model saved in gdrive, replace this with the full path of model in gdrive
  if 'pipe' not in locals():
    scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
    pipe = StableDiffusionPipeline.from_pretrained(model_path, scheduler=scheduler, safety_checker=None, torch_dtype=torch.float16).to("cuda")
    g_cuda = None

  g_cuda = torch.Generator(device='cuda')

  g_cuda.manual_seed(seed)

  with autocast("cuda"), torch.inference_mode():
    images = pipe(
        prompt,
        height=height,
        width=width,
        negative_prompt=negative_prompt,
        num_images_per_prompt=num_samples,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=g_cuda
    ).images[0]
    images.save("./response.jpg")

  return send_file("./response.jpg", mimetype='image/jpg')



@app.route('/text_prompt', methods = ['POST'])
def text_prompt():
  file = request.files['image']
  prompt = request.form['text']
  img = Image.open(file.stream)
  with open('filename.txt', 'w') as f:
    f.write(prompt)

  model_id = "runwayml/stable-diffusion-v1-5"
  pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, revision="fp16")
  pipe = pipe.to("cuda")

  image = pipe(prompt).images[0]
  image.save("./response.jpg")

  return send_file("./response.jpg", mimetype='image/jpg')

@app.route('/guidance', methods = ['POST'])
def guidance_process():
  prompt = request.form['text']

  def blue_loss(image):
    code_string = request.form['code']
    return eval(code_string)
  

  # Prep text 
  text_input = tokenizer([prompt], padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
  with torch.no_grad():
    text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]

  # And the uncond. input as before:
  max_length = text_input.input_ids.shape[-1]
  uncond_input = tokenizer(
    [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
  )
  with torch.no_grad():
    uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0] 
  text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

  # Prep Scheduler
  scheduler.set_timesteps(num_inference_steps)

  # Prep latents
  latents = torch.randn(
    (batch_size, unet.in_channels, height // 8, width // 8),
    generator=generator,
    )
  latents = latents.to(torch_device)
  latents = latents * scheduler.init_noise_sigma

  # Loop
  for i, t in tqdm(enumerate(scheduler.timesteps)):
    # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
    latent_model_input = torch.cat([latents] * 2)
    sigma = scheduler.sigmas[i]
    latent_model_input = scheduler.scale_model_input(latent_model_input, t)

    # predict the noise residual
    with torch.no_grad():
        noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings)["sample"]

    # perform CFG
    noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
    noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)
    
    #### ADDITIONAL GUIDANCE ###
    if i%5 == 0:
        # Requires grad on the latents
        latents = latents.detach().requires_grad_()

        # Get the predicted x0:
        # latents_x0 = latents - sigma * noise_pred
        latents_x0 = scheduler.step(noise_pred, t, latents).pred_original_sample

        # Decode to image space
        denoised_images = vae.decode((1 / 0.18215) * latents_x0).sample / 2 + 0.5 # range (0, 1)

        # Calculate loss
        loss = blue_loss(denoised_images) * blue_loss_scale
        
        # Occasionally print it out
        if i%10==0:
            print(i, 'loss:', loss.item())

        # Get gradient
        cond_grad = torch.autograd.grad(loss, latents)[0]

        # Modify the latents based on this gradient
        latents = latents.detach() - cond_grad * sigma**2
    
    # Now step with scheduler
    latents = scheduler.step(noise_pred, t, latents).prev_sample


  latents_to_pil(latents)[0].save("./response.jpg")

  return send_file("./response.jpg", mimetype='image/jpg')

@app.route('/imgtoimgpipe', methods = ['POST'])
def imgtoimgpipe():
  prompt = request.form['text']
  file = request.files['image']
  img = Image.open(file.stream)

  controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)
  pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
    )
  
  pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
  pipe.enable_model_cpu_offload()
  pipe.enable_xformers_memory_efficient_attention()

  generator = torch.Generator(device="cpu").manual_seed(2)

  output = pipe(
    prompt,
    img,
    negative_prompt="monochrome, lowres, bad anatomy, worst quality, low quality",
    generator=generator,
    num_inference_steps=20,
    )

  output.images[0].save("./response.jpg")

  return send_file("./response.jpg", mimetype='image/jpg')

@app.route('/', methods = ['GET'])
def get_func():
  return "Hello Friends! Thank you! for reading this code."

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://632b-34-172-43-2.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


0it [00:00, ?it/s]

ERROR:__main__:Exception on /guidance [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/flask/app.py", line 2528, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.9/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.9/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.9/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-14-c5ce71048662>", line 127, in guidance_process
    loss = blue_loss(denoised_images) * blue_loss_scale
  File "<ipython-input-14-c5ce71048662>", line 71, in blue_loss
    return eval(code_string)
  File "<string>", line 1, in <module>
NameError: name 'img' is not defined
INFO:werkzeug:127.0.0.1 - - [19/Apr/2023 07:03:41] "POST

...

0 loss: 154.57272338867188
10 loss: 31.069385528564453
20 loss: 27.31328010559082
30 loss: 23.140625
40 loss: 23.035144805908203


INFO:werkzeug:127.0.0.1 - - [19/Apr/2023 07:38:51] "POST /guidance HTTP/1.1" 200 -


0it [00:00, ?it/s]

0 loss: 130.21939086914062
10 loss: 41.85336685180664
20 loss: 29.547517776489258
30 loss: 24.370647430419922
40 loss: 23.663339614868164


INFO:werkzeug:127.0.0.1 - - [19/Apr/2023 07:38:57] "POST /guidance HTTP/1.1" 200 -


In [ ]:
# !pip install urllib2
import urllib, cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
url = "https://cdn.britannica.com/77/196577-050-1101EEBD/Michelangelos-David-Goliath-one-statues-world.jpg"

In [ ]:
req = urllib.request.urlopen(url)
arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
img = cv2.imdecode(arr, -1) # 'Load it as it is'
img_copy = img.copy()

In [ ]:
plt.imshow(img)

In [ ]:
img_copy[150:1100, 400:800] = 255

In [ ]:
plt.imshow(img_copy)

In [ ]:
image_lst = []
for i in range(255, 0, -10):
  img_copy = img.copy()
  img_copy[150:1100, 400:800] = img_copy[150:1100, 400:800] + i#np.random.normal(0, i)
  image_lst.append(img_copy)

In [ ]:
import imageio, cv2

In [ ]:
imageio.mimsave('./video.gif', image_lst, fps=10)

In [ ]:
image_lst = []
def fadeIn(img1, img2, len=30): #pass images here to fade between
    global img
    while True:
        for IN in range(0,len):
            fadein = IN/float(len)
            dst = cv2.addWeighted( img1, 1-fadein, img2, fadein, 0)
            temp = img.copy()/255.
            temp[150:1100, 400:800] = dst
            image_lst.append(temp)
            print(fadein)
        break

In [ ]:
fadeIn(img[150:1100, 400:800]/255., np.ones((917, 400, 3)))